In [1]:
# Imports

import pandas as pd
import numpy as np
import openpyxl

### Read the first data file

In [2]:
# Reading the files, Omit the first 3 rows and last 9 rows of unnecessary data

data1 = pd.read_csv("../data/raw/summary-statistics.csv", skiprows=3)[:-9]
data1.dropna(inplace = True)

In [3]:
data1['Total male population (000s)'] = data1['Total male population (000s)'].apply(lambda x: x.replace(',', ''))
data1['Total female population (000s)'] = data1['Total female population (000s)'].apply(lambda x: x.replace(',', ''))
data1['Total population (000s)'] = data1['Total population (000s)'].apply(lambda x: x.replace(',', ''))
data1['Median age at death'] = data1['Median age at death'].apply(lambda x: x.replace(',', ''))
data1['Total fertility'] = data1['Total fertility'].apply(lambda x: x.replace(',', ''))

In [4]:
data1['Median age at death'] = data1['Median age at death'].apply(lambda x: x.replace('--', '0'))
data1['Total fertility'] = data1['Total fertility'].apply(lambda x: x.replace('--', '0'))

In [5]:
data1 = data1.astype({'Total male population (000s)':'float',
                                          'Total female population (000s)':'float',
                                          'Total population (000s)':'float',
                                          'Median age at death':'float',
                                          'Total fertility':'float',
                                         })

In [6]:
data1['Year'] = data1['Year'].astype('str')

In [7]:
# Exporting to csv file for plotting
data1.to_csv('../data/clean/population_trends.csv', index=False)

In [8]:
# Creating a tidy dataframe for viewing the children dependency, elderly denpendency
dependency_melt = pd.melt(data1[['Year', 'Child dependency', 'Elderly dependency', 'Total dependency']], id_vars = ['Year'], value_vars= ['Child dependency', 'Elderly dependency', 'Total dependency'], var_name= 'Dependency', value_name= 'Value')

In [9]:
# Exporting to csv file for plotting
dependency_melt.to_csv('../data/clean/dependency.csv', index=False)

### Read the second data file

In [10]:
data2 = pd.read_csv("../data/raw/provincial_voter_participation_by_age_group.csv")

In [11]:
data2['PARTICIPATION'] = data2['PARTICIPATION'].apply(lambda x: x.replace(',', ''))
data2['REGISTERED_VOTERS'] = data2['REGISTERED_VOTERS'].apply(lambda x: x.replace(',', ''))

In [12]:
data2['Year'] = data2['EVENT_YEAR'].astype('str')

In [13]:
data2 = data2.astype({'PARTICIPATION':'int',
                                          'REGISTERED_VOTERS':'int',
                                          'AGE_GROUP':'str',
                                                  'Year': 'str',})

In [14]:
data2 = data2[['Year', 'ED_NAME', 'AGE_GROUP', 'PARTICIPATION', 'REGISTERED_VOTERS']]

In [15]:
data2['RATIO_PARTICIPATION'] = round(data2['PARTICIPATION']/data2['REGISTERED_VOTERS'],2)

In [16]:
# Exporting to csv file for plotting
data2.to_csv('../data/clean/participation_by_district.csv')

In [17]:
participation_df = data2.groupby(['Year', 'AGE_GROUP'])['PARTICIPATION', 'REGISTERED_VOTERS'].sum().reset_index()

C:\Users\ranji\AppData\Local\Temp\ipykernel_18052\1055910838.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  participation_df = data2.groupby(['Year', 'AGE_GROUP'])['PARTICIPATION', 'REGISTERED_VOTERS'].sum().reset_index()


In [18]:
participation_df['RATIO_PARTICIPATION'] = round(participation_df['PARTICIPATION']/participation_df['REGISTERED_VOTERS'],2)

In [19]:
# Exporting to csv file for plotting
participation_df.to_csv('../data/clean/participation_aggregate.csv', index=False)

### Read the third data file

In [20]:
# Reading the relevant rows that provide the public sectore expenditure on health in British Columbia

data3 = pd.read_excel('../data/raw/nhex-series-B-2022-en.xlsx', sheet_name='Public', skiprows=4,  nrows= 46)

In [21]:
# Removing the spaces from the column names
data3.columns = data3.columns.str.strip()

In [22]:
data3['Year'] = data3['Year'].astype(str)

In [23]:
# Selecting only the columns of interest to us
healthcare_exp_df = data3[['Year', 'B.C.']]

In [24]:
healthcare_exp_df.dropna(inplace = True)

C:\Users\ranji\AppData\Local\Temp\ipykernel_18052\943563905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  healthcare_exp_df.dropna(inplace = True)


### Per-capita health expenditure computation

In [25]:
# Joining the data to compute the per-capita health expenditure

healthcare_exp_df = pd.merge(healthcare_exp_df, data1, how = 'left', on = 'Year')

In [26]:
# Numerator is in millions, denominator is in thousands
healthcare_exp_df['Per_capita_healthexp'] = healthcare_exp_df['B.C.'] / healthcare_exp_df['Total population (000s)']

In [27]:
# Multiply the result with 1000 to get in CAD.
healthcare_exp_df['Per_capita_healthexp']  = healthcare_exp_df['Per_capita_healthexp'] * 1000

In [28]:
healthcare_exp_df = healthcare_exp_df[['Year', 'Per_capita_healthexp', 'Median age']]

In [29]:
# Exporting to csv file for plotting
healthcare_exp_df.to_csv('../data/clean/healthcare_expenses.csv', index = False)